In [1]:
import numpy as np
import pyrealsense2 as rs
import matplotlib.pyplot as plt     
from datetime import datetime
from mmwave.dataloader import DCA1000
from mmwave import dsp

from IPython.display import clear_output
import winsound
duration = 1500  # milliseconds
freq = 440  # Hz
import time
from mmwave.dsp.utils import Window
import traceback
import PyQt5

In [2]:
NUM_TX = 3
NUM_RX = 4
CHIRP_LOOPS = 128
NUM_ADC_SAMPLES = 149
numDopplerBins = 128
numRangeBins = 149

# Data sampling configuration
c = 3e8 # Speed of light (m/s)
sample_rate = 5000 # Rate at which the radar samples from ADC (ksps - kilosamples per second)
freq_slope = 60.06 # Frequency slope of the chirp (MHz/us)
adc_samples = 256 # Number of samples from a single chirp

start_freq = 77.0 # Starting frequency of the chirp (GHz)
idle_time = 7 # Time before starting next chirp (us)
ramp_end_time = 58.57 # Time after sending each chirp (us)

# Range resolution
range_resolution = (c * sample_rate * 1e3) / (2 * freq_slope * 1e12 * NUM_ADC_SAMPLES)
print(f'Range Resolution: {range_resolution} [meters]')

# Maximum Range
max_range = (c * sample_rate * 1e3) / (2 * freq_slope * 1e12)
print(f'Maximum Range: {max_range} [meters]')

# Make sure your equation translates to the following
velocity_res = c / (2 * start_freq * 1e9 * (idle_time + ramp_end_time) * 1e-6 * CHIRP_LOOPS * NUM_TX)
print(f'Velocity Resolution: {velocity_res} [meters/second]')

ranges = np.arange(adc_samples) * range_resolution

dca = DCA1000()

Range Resolution: 0.050285285184614045 [meters]
Maximum Range: 7.492507492507492 [meters]
Velocity Resolution: 0.07736849089601874 [meters/second]


In [3]:
def mmwave_data_collection():
    i = 0
    radar_list = []
    time_list = []
    
    winsound.Beep(freq, duration)
    while True:
        i = i + 1
        try:
            adc_data = dca.read()
            frame3d = dca.organize(adc_data, CHIRP_LOOPS*NUM_TX, NUM_RX, NUM_ADC_SAMPLES, model='1443') 
            # (384,4,149) (num_chirps_per_frame, num_rx_antennas, num_adc_samples)
            print(frame3d.shape)
            radar_list.append(adc_data)

            # (2) Range Processing
        
            radar_cube = dsp.range_processing(frame3d, window_type_1d=Window.BLACKMAN)
            #--> output (num_chirps_per_frame, num_rx_antennas, num_range_bins)
            print(radar_cube.shape)

            det_matrix, aoa_input = dsp.doppler_processing(radar_cube, num_tx_antennas=3, clutter_removal_enabled=True)
            #(numRangeBins, num_doppler_bins) and  (numRangeBins, numVirtualAntennas, num_doppler_bins)
        # --- CFAR, SNR is calculated as well.
            fft2d_sum = det_matrix.astype(np.int64)
            thresholdDoppler, noiseFloorDoppler = np.apply_along_axis(func1d=dsp.ca_,
                                                                  axis=0,
                                                                  arr=fft2d_sum.T,
                                                                  l_bound=1.5,
                                                                  guard_len=4,
                                                                  noise_len=16)

            thresholdRange, noiseFloorRange = np.apply_along_axis(func1d=dsp.ca_,
                                                              axis=0,
                                                              arr=fft2d_sum,
                                                              l_bound=2.5,
                                                              guard_len=4,
                                                              noise_len=16)

            thresholdDoppler, noiseFloorDoppler = thresholdDoppler.T, noiseFloorDoppler.T
            det_doppler_mask = (det_matrix > thresholdDoppler)
            det_range_mask = (det_matrix > thresholdRange)

            # Get indices of detected peaks
            full_mask = (det_doppler_mask & det_range_mask)
            det_peaks_indices = np.argwhere(full_mask == True)

        # peakVals and SNR calculation
            peakVals = fft2d_sum[det_peaks_indices[:, 0], det_peaks_indices[:, 1]]
            snr = peakVals - noiseFloorRange[det_peaks_indices[:, 0], det_peaks_indices[:, 1]]

            dtype_location = '(' + str(NUM_TX) + ',)<f4'
            dtype_detObj2D = np.dtype({'names': ['rangeIdx', 'dopplerIdx', 'peakVal', 'location', 'SNR'],
                                   'formats': ['<i4', '<i4', '<f4', dtype_location, '<f4']})
            detObj2DRaw = np.zeros((det_peaks_indices.shape[0],), dtype=dtype_detObj2D)
            detObj2DRaw['rangeIdx'] = det_peaks_indices[:, 0].squeeze()
            detObj2DRaw['dopplerIdx'] = det_peaks_indices[:, 1].squeeze()
            detObj2DRaw['peakVal'] = peakVals.flatten()
            detObj2DRaw['SNR'] = snr.flatten()

            # Further peak pruning. This increases the point cloud density but helps avoid having too many detections around one object.
            detObj2DRaw = dsp.prune_to_peaks(detObj2DRaw, det_matrix, numDopplerBins, reserve_neighbor=True)

            # --- Peak Grouping
            detObj2D = dsp.peak_grouping_along_doppler(detObj2DRaw, det_matrix, numDopplerBins)
            SNRThresholds2 = np.array([[2, 23], [10, 11.5], [35, 16.0]])
            peakValThresholds2 = np.array([[4, 275], [1, 400], [500, 0]])
            detObj2D = dsp.range_based_pruning(detObj2D, SNRThresholds2, peakValThresholds2, numRangeBins, 0.5, range_resolution)
            #Need to identify the available thresholds

            azimuthInput = aoa_input[detObj2D['rangeIdx'], :, detObj2D['dopplerIdx']]
            #Check the dimensions of this input

            x, y, z = dsp.naive_xyz(azimuthInput.T)
            xyzVecN = np.zeros((3, x.shape[0]))
            xyzVecN[0] = x * range_resolution * detObj2D['rangeIdx']
            xyzVecN[1] = y * range_resolution * detObj2D['rangeIdx']
            xyzVecN[2] = z * range_resolution * detObj2D['rangeIdx']

            
            clear_output(wait=True)
            fig = plt.figure(figsize=(20,20))

            ax1 = fig.add_subplot(121, projection='3d')
            ax2 = fig.add_subplot(122)

            ax1.set_title("3D Scatter Plot")
            ax1.scatter(xyzVecN[0], xyzVecN[1], xyzVecN[2])
            ax1.set_xlabel('Azimuth(m)')
            ax1.set_ylabel('Range(m)')
            ax1.set_zlabel('Elevation(m)')

            # xyzVec = xyzVecN[:, (np.abs(xyzVecN[2]) < 1.5)]
            # xyzVecN = xyzVecN[:, (np.abs(xyzVecN[2]) < 1.5)]
            # ax2.set_ylim(bottom=0, top=10)
            # ax2.set_ylabel('Range')
            # ax2.set_xlim(left=-4, right=4)
            # ax2.set_xlabel('Azimuth')
            # ax2.grid(b=True)

            plt.show()

            datetime_object = datetime.now()
            time_list.append(datetime_object)
            print(datetime.now(), "Done frame", i)
        except Exception as e:
            print(e)
            traceback.print_exc()
            print("Timeout occured")
            arr_1 = np.asarray(radar_list)
            arr_2 = np.asarray(time_list)
            print(arr_1.shape)
            print("Saving")
            np.savez("radar_cam_100_frames.npz",arr_1,arr_2)
            break;
        

In [4]:
mmwave_data_collection()

timed out
Timeout occured
(0,)
Saving


Traceback (most recent call last):
  File "C:\Users\z5262974\AppData\Local\Temp/ipykernel_16628/1093599558.py", line 10, in mmwave_data_collection
    adc_data = dca.read()
  File "e:\OneDrive - UNSW\PhD\Experiments\2021-06\openradar\mmwave\dataloader\adc.py", line 176, in read
    packet_num, byte_count, packet_data = self._read_data_packet()
  File "e:\OneDrive - UNSW\PhD\Experiments\2021-06\openradar\mmwave\dataloader\adc.py", line 233, in _read_data_packet
    data, addr = self.data_socket.recvfrom(MAX_PACKET_SIZE)
socket.timeout: timed out


In [5]:
loaded_mmwave = np.load("radar_cam_100_frames.npz",allow_pickle=True)
print(loaded_mmwave["arr_0"].shape)
print(loaded_mmwave["arr_1"].shape)

(0,)
(0,)


In [6]:
#%matplotlib qt
%matplotlib inline

In [1]:
mmwave_adc = loaded_mmwave["arr_0"]

for i in mmwave_adc:
    clear_output(wait=True)
    frame3d = dca.organize(i, CHIRP_LOOPS*NUM_TX, NUM_RX, NUM_ADC_SAMPLES, model='1443') 
    # (384,4,149) (num_chirps_per_frame, num_rx_antennas, num_adc_samples)
    print(frame3d.shape)

    # (2) Range Processing

    radar_cube = dsp.range_processing(frame3d, window_type_1d=Window.BLACKMAN)
    #--> output (num_chirps_per_frame, num_rx_antennas, num_range_bins)
    print(radar_cube.shape)

    det_matrix, aoa_input = dsp.doppler_processing(radar_cube, num_tx_antennas=3, clutter_removal_enabled=True)
    #(numRangeBins, num_doppler_bins) and  (numRangeBins, numVirtualAntennas, num_doppler_bins)
# --- CFAR, SNR is calculated as well.
    fft2d_sum = det_matrix.astype(np.int64)
    thresholdDoppler, noiseFloorDoppler = np.apply_along_axis(func1d=dsp.ca_,
                                                            axis=0,
                                                            arr=fft2d_sum.T,
                                                            l_bound=1.5,
                                                            guard_len=4,
                                                            noise_len=16)

    thresholdRange, noiseFloorRange = np.apply_along_axis(func1d=dsp.ca_,
                                                        axis=0,
                                                        arr=fft2d_sum,
                                                        l_bound=2.5,
                                                        guard_len=4,
                                                        noise_len=16)

    thresholdDoppler, noiseFloorDoppler = thresholdDoppler.T, noiseFloorDoppler.T
    det_doppler_mask = (det_matrix > thresholdDoppler)
    det_range_mask = (det_matrix > thresholdRange)

    # Get indices of detected peaks
    full_mask = (det_doppler_mask & det_range_mask)
    det_peaks_indices = np.argwhere(full_mask == True)

# peakVals and SNR calculation
    peakVals = fft2d_sum[det_peaks_indices[:, 0], det_peaks_indices[:, 1]]
    snr = peakVals - noiseFloorRange[det_peaks_indices[:, 0], det_peaks_indices[:, 1]]

    dtype_location = '(' + str(NUM_TX) + ',)<f4'
    dtype_detObj2D = np.dtype({'names': ['rangeIdx', 'dopplerIdx', 'peakVal', 'location', 'SNR'],
                            'formats': ['<i4', '<i4', '<f4', dtype_location, '<f4']})
    detObj2DRaw = np.zeros((det_peaks_indices.shape[0],), dtype=dtype_detObj2D)
    detObj2DRaw['rangeIdx'] = det_peaks_indices[:, 0].squeeze()
    detObj2DRaw['dopplerIdx'] = det_peaks_indices[:, 1].squeeze()
    detObj2DRaw['peakVal'] = peakVals.flatten()
    detObj2DRaw['SNR'] = snr.flatten()

    # Further peak pruning. This increases the point cloud density but helps avoid having too many detections around one object.
    detObj2DRaw = dsp.prune_to_peaks(detObj2DRaw, det_matrix, numDopplerBins, reserve_neighbor=True)

    # --- Peak Grouping
    detObj2D = dsp.peak_grouping_along_doppler(detObj2DRaw, det_matrix, numDopplerBins)
    SNRThresholds2 = np.array([[0,10], [0,10], [0, 10]])
    peakValThresholds2 = np.array([[4, 100], [1, 100], [1,100]])
    detObj2D = dsp.range_based_pruning(detObj2D, SNRThresholds2, peakValThresholds2, numRangeBins, 0.0, range_resolution)
    #Need to identify the available thresholds

    azimuthInput = aoa_input[detObj2D['rangeIdx'], :, detObj2D['dopplerIdx']]
    #Check the dimensions of this input

    x, y, z = dsp.naive_xyz(azimuthInput.T)
    xyzVecN = np.zeros((3, x.shape[0]))
    xyzVecN[0] = x * range_resolution * detObj2D['rangeIdx']
    xyzVecN[1] = y * range_resolution * detObj2D['rangeIdx']
    xyzVecN[2] = z * range_resolution * detObj2D['rangeIdx']

    fig = plt.figure(figsize=(15,5))
     
    
    ax1 = fig.add_subplot(131, projection='3d')
    ax2 = fig.add_subplot(132)
    ax3 = fig.add_subplot(133)

    ax1.set_title("3D Scatter Plot")
    ax1.scatter(xyzVecN[0], xyzVecN[1], xyzVecN[2],s=10, marker='o')
    ax1.set_xlabel('Azimuth(m)')
    ax1.set_ylabel('Range(m)')
    ax1.set_zlabel('Elevation(m)')
    ax1.set_xlim(left=-4, right=4)
    ax1.set_ylim(bottom=0, top=8)
    ax1.set_zlim(top=-4, bottom=4)
    ax1.grid(b=True)

    #xyzVec = xyzVecN[:, (np.abs(xyzVecN[2]) < 1.5)]
    ax2.set_title("Range Azimuth Plot")
    ax2.set_ylim(bottom=0, top=8)
    ax2.set_ylabel('Range(m)')
    ax2.set_xlim(left=-4, right=4)
    ax2.set_xlabel('Azimuth(m)')
    ax2.grid(b=True)
    ax2.scatter(xyzVecN[0], xyzVecN[1])

    ax3.set_title("Range Elevation Plot")
    ax3.set_ylim(bottom=0, top=8)
    ax3.set_ylabel('Range(m)')
    ax3.set_xlim(left=-4, right=4)
    ax3.set_xlabel('Elevation(m)')
    ax3.grid(b=True)
    ax3.scatter(xyzVecN[0], xyzVecN[2])

    fig.tight_layout()
    plt.show()
    datetime_object = datetime.now()
    print(datetime.now(), "Done frame", i)
    break
    #plt.pause(2)

NameError: name 'loaded_mmwave' is not defined